In [16]:
from utils.data_handling import *
from utils.sensor_optimisation_tsvd import *
from parameters import *
import GPy
import scipy.stats
import time

import matplotlib.pyplot as plt
np.random.seed(101)



### Importing the Tracer Data to Select the Optimisation set S

In [2]:
parameters['field_name'] = "Tracer"
parameters


{'i_start': 0,
 'i_end': 988,
 'crop': None,
 'field_name': 'Tracer',
 'cov_method': 'sample'}

In [3]:
loaded = initial_load_data(parameters, recompute=False)
ref_vtu, data_df, loc_df, time_df = loaded

100%|██████████| 1/1 [00:00<00:00,  6.41it/s]

### Loading files from original VTU
==> Import vtu files from 0 to 0


Number of Locations after cropping :  100040
==> Loading from : ../data/temp_data/cache_0_988_None/loc_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/time_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/data_Tracer.pkl


### Working subset of the data : set S

In [4]:
S_human = human_level_subset(buildingshape, loc_df, 2,1 )
print("The remaining number of points is : ", str(len(S_human)))

The remaining number of points is :  37847


In [5]:
S_working = working_subset(data_df, loc_df, nbins = (25,25,25), threshold_sum = 10**-2 )
print("The remaining number of points is : ", str(len(S_working)))

The remaining number of points is :  57725


In [6]:
S = np.intersect1d(S_human,S_working)
print("The remaining number of points is : ", str(len(S)))

The remaining number of points is :  23643


In [7]:
#A_list = set_to_onehot(S,data_df.shape[0])
#save_vtu(ref_vtu[0],'FinalSelection',A_list)

### Import the Field of interest for computing the optimisation : TracerBackground / Pressure

In [8]:
parameters['i_end'] = 988
parameters['field_name'] = "Tracer"
parameters


{'i_start': 0,
 'i_end': 988,
 'crop': None,
 'field_name': 'Tracer',
 'cov_method': 'sample'}

In [9]:
loaded = initial_load_data(parameters, recompute=False)
ref_vtu, data_df, loc_df, time_df = loaded

100%|██████████| 1/1 [00:00<00:00,  6.59it/s]

### Loading files from original VTU
==> Import vtu files from 0 to 0


Number of Locations after cropping :  100040
==> Loading from : ../data/temp_data/cache_0_988_None/loc_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/time_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/data_Tracer.pkl


In [10]:
X = loc_df.values[:,:] #[I,:dim]
Z = data_df.values[:,:]#[:,t].reshape(-1,1) # [I,t].reshape(-1,1)
Z.shape

(100040, 989)

### Detrending Data : 
Plotting the data along each direction in order to see if there is a obvious trend. It seams that it is not the case. 

In [11]:
Z[S,:] = (Z[S,:]  - Z[S,:].mean(axis=1,keepdims=True))

### Sensor Optimisation with TSVD : 


Define the Sets for the optimisation

In [12]:
sets = define_sets(S)

Number of sensors to place : 

In [13]:
k = 10

Truncation parameteter for the TSVD : 

In [18]:
tau = 25

In [19]:
A_opt = {}

In [ ]:
start = time.time()
A_opt['lazy'] = sensor_loc_optimisation_lazy(k,Z, sets, tau)
end = time.time()

Inner Loop:   1%|          | 237/23643 [02:41<4:38:52,  1.40it/s]


Inner Loop:   1%|▏         | 311/23643 [03:33<4:50:05,  1.34it/s]

In [ ]:
A_opt['naive'] = sensor_loc_optimisation_naive(k,Z, sets, tau)

In [32]:
A_opt['lazy'] 

array([38726, 14276, 91348,  5338, 40994, 29626, 65851, 65734,   851,
        2293])

In [30]:
print(loc_df.loc[A_opt['lazy'] ,:].T.to_latex(float_format="%.2f"))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  38726 &  14276 &  91348 &  5338  &  40994 &  29626 &  65851 &  65734 &  851   &  2293  \\
\midrule
X & -50.50 &  35.63 &  43.05 & 137.58 &  80.34 &  27.79 &  49.54 &  54.59 &  77.54 &  62.03 \\
Y &  48.85 &  58.69 &  27.51 &  55.78 &  76.20 &  26.24 &  28.88 &  25.42 &  34.82 &  41.92 \\
Z &  14.84 &   0.20 &   7.76 &   0.20 &   0.20 &  12.06 &  17.65 &   3.80 &   0.20 &   0.20 \\
\bottomrule
\end{tabular}



In [23]:
end - start 

18083.875992059708

In [28]:
A_list = set_to_onehot(A_opt['lazy'] ,data_df.shape[0])
save_vtu(ref_vtu[0],'Opt_TSVD_full_tau25',A_list)

==> Saved to : ../data/temp_data/LSBU_res_2019:08:15-09:49:23_Opt_TSVD_full_tau25.vtu


In [37]:
np.savetxt('Optimal Points/localpoints_TSVD25',A_opt['lazy'])